In [1]:
import pandas as pd

**Главные задачи, которые должен выполнять скрипт:** <br>
Извлекать реплики с приветствием – где менеджер поздоровался. <br>
Извлекать реплики, где менеджер представил себя. <br>
Извлекать имя менеджера. <br>
Извлекать название компании. <br>
Извлекать реплики, где менеджер попрощался. <br>
Проверять требование к менеджеру: «В каждом диалоге обязательно необходимо поздороваться и попрощаться с клиентом»

In [2]:
df = pd.read_csv('test_data.csv')
df

,dlg_id,line_n,role,text
0,0,0,client,Алло
1,0,1,manager,Алло здравствуйте
2,0,2,client,Добрый день
3,0,3,manager,Меня зовут ангелина компания диджитал бизнес з...
4,0,4,client,Ага
...,...,...,...,...
475,5,138,manager,По поводу виджетов и с ними уже обсудите конкр...
476,5,139,manager,Все я вам высылаю счет и с вами на связи если ...
477,5,140,client,Спасибо спасибо
478,5,141,client,Да да тогда созвонимся ага спасибо вам давайте


## Извлекать реплики с приветствием – где менеджер поздоровался

In [3]:
df1 = df[(df['role']=='manager')]
df2=df1[df1['text'].str.contains('здравствуйте|добрый|добрый день|доброе утро|добрый вечер', case=False, na=False)] 
df2.text.tolist()

['Алло здравствуйте',
 'Алло здравствуйте',
 'Алло здравствуйте',
 'Алло дмитрий добрый день',
 'Добрый меня максим зовут компания китобизнес удобно говорить']

## Извлекать реплики, где менеджер представил себя

In [4]:
df1 = df[(df['role']=='manager')]
df2 = df1[df1['text'].str.contains('меня зовут|моё имя|мое имя|зовут', case=False, na=False)]
df2.text.tolist()

['Меня зовут ангелина компания диджитал бизнес звоним вам по поводу продления лицензии а мы с серым у вас скоро срок заканчивается',
 'Меня зовут ангелина компания диджитал бизнес звоню вам по поводу продления а мы сели обратила внимание что у вас срок заканчивается',
 'Меня зовут ангелина компания диджитал бизнес звоню вам по поводу продления лицензии а мастера мы с вами сотрудничали по видео там',
 'Добрый меня максим зовут компания китобизнес удобно говорить']

## Извлекать имя менеджера

In [5]:
def get_names(all_list):
    names = []
    for i in all_list: # проходим по всем строкам общего списка
        L = i.split() # берем каждую строку и делим ее на отдельные слова
        L2 = [] # вспомогательный список
        
        #1 проверка, шаблон: меня зовут _____, мое имя ____ и т.д
        for j in range(len(L)-1): 
            if L[j] in ('меня', 'моё', 'мое'): 
                if L[j+1] in ('зовут', 'имя'):
                    L2 = L[j+2:]
                    if len(L2) !=0: #проверка списка на пустоту
                        names.append(L2[0])
                    break
        #2 проверка, шаблон: меня _____ зовут
        for j in range(len(L)-2):
            if L[j] == 'меня': 
                if L[j+2] == 'зовут':
                    L2 = L[j+1:]
                    if len(L2) !=0: #проверка списка на пустоту
                        names.append(L2[0])
                    break
            
    return set(names) # фильтруем на уникальные имена
        
df1 = df[(df['role']=='manager')] #все фразы менеджера
df2 = df1[df1['text'].str.contains('меня зовут|моё имя|мое имя|зовут', case=False, na=False)] # все строки где имеется имя менеджера
print('Имена менеджеров:', *get_names(df2['text'].str.lower().tolist())) # список строк в нижнем регистре передаем в функцию


Имена менеджеров: максим ангелина


## Извлекать название компании

In [6]:
def get_company_names(list_str):
    comp_names = []
    for i in list_str:
        L = i.split()  
        ind_sreza = L.index('компания')
        comp_names.append(' '.join(L[ind_sreza+1:ind_sreza+3]))
    return list(set(comp_names))

df3 = df[(df['text'].str.contains('компания', case=False, na=False)) & (df['role']=='manager') ]
rez = get_company_names(df2.text.str.lower().tolist())
print('Потенциальные названия компаний: ', rez)

rez=['китобизнес' if i=='китобизнес удобно' else i for i in rez]
print('Названия компаний:', rez)

Потенциальные названия компаний:  ['диджитал бизнес', 'китобизнес удобно']
Названия компаний: ['диджитал бизнес', 'китобизнес']


## Извлекать реплики, где менеджер попрощался

In [7]:
df1 = df[(df['role']=='manager')]
df2=df1[df1['text'].str.contains('до свидания|всего доброго', case=False, na=False)]
df2.text.tolist()

['Всего хорошего до свидания',
 'До свидания',
 'Угу все хорошо да понедельника тогда всего доброго',
 'Во вторник все ну с вами да тогда до вторника до свидания',
 'Ну до свидания хорошего вечера']

## Проверять требование к менеджеру: «В каждом диалоге обязательно необходимо поздороваться и попрощаться с клиентом»

In [8]:
# Шаг 1. Фильтруем данные
df1 = df[(df['role']=='manager')]
df2=df1[df1['text'].str.contains('здравствуйте|добрый|добрый день|доброе утро|добрый вечер|до свидания|всего доброго', case=False, na=False)] 

In [9]:
# Шаг 2. Добавляем столбец "greetings" - является ли фраза приветствием (для наглядности)
L = df2.text.str.contains('здравствуйте|добрый|добрый день|доброе утро|добрый вечер', case=False)
df2.insert (4, 'greetings', L)
df2

,dlg_id,line_n,role,text,greetings
1,0,1,manager,Алло здравствуйте,True
108,0,108,manager,Всего хорошего до свидания,False
110,1,1,manager,Алло здравствуйте,True
163,1,54,manager,До свидания,False
166,2,2,manager,Алло здравствуйте,True
250,3,1,manager,Алло дмитрий добрый день,True
251,3,2,manager,Добрый меня максим зовут компания китобизнес у...,True
300,3,51,manager,Угу все хорошо да понедельника тогда всего доб...,False
335,4,33,manager,Во вторник все ну с вами да тогда до вторника ...,False
479,5,142,manager,Ну до свидания хорошего вечера,False


In [10]:
# Шаг 3. Проверка требования «В каждом диалоге обязательно необходимо поздороваться и попрощаться с клиентом»
L_grt = df2.text.str.contains('здравствуйте|добрый|добрый день|доброе утро|добрый вечер', case=False).tolist()
L_grt_status = []
fl_cont = False # флаг чтобы пропускать следующий цикл
for i in range(len(L_grt)):
    
    # пропуск шага если было приветствие и прощание
    if fl_cont:
        fl_cont = False
        continue
    
    # проверка последнего шага
    if i == len(L_grt)-1:
        if L_grt[i] == True:
            L_grt_status.append('Менеджер поздоровался, но не попрощался')
        else:
            L_grt_status.append('Менеджер попрощался, но не поздороваться')
        break
    
    # проверка: было приветствие и прощание
    if L_grt[i] == True and L_grt[i+1] == False: 
        L_grt_status.append('ОК')
        L_grt_status.append('ОК')
        fl_cont = True # пропуск следующего цикла
        continue
    
    # проверка: было только прощание
    if L_grt[i] == False and L_grt[i+1] == True: 
        L_grt_status.append('Менеджер попрощался, но не поздороваться')
    
    # проверка: было только приветствие
    if L_grt[i] == True and L_grt[i+1] == True:
        L_grt_status.append('Менеджер поздоровался, но не попрощался')
        
    # проверка: было только прощание
    if L_grt[i] == False and L_grt[i+1] == False:
        L_grt_status.append('Менеджер попрощался, но не поздороваться')
       
L_grt_status

['ОК',
 'ОК',
 'ОК',
 'ОК',
 'Менеджер поздоровался, но не попрощался',
 'Менеджер поздоровался, но не попрощался',
 'ОК',
 'ОК',
 'Менеджер попрощался, но не поздороваться',
 'Менеджер попрощался, но не поздороваться']

In [11]:
# Шаг 4. Выводим результат списка L_grt_status в новый сталбец status
S = pd.Series(L_grt_status,
             index=df2.index)
df2.insert (5, 'status', S)
df2

,dlg_id,line_n,role,text,greetings,status
1,0,1,manager,Алло здравствуйте,True,ОК
108,0,108,manager,Всего хорошего до свидания,False,ОК
110,1,1,manager,Алло здравствуйте,True,ОК
163,1,54,manager,До свидания,False,ОК
166,2,2,manager,Алло здравствуйте,True,"Менеджер поздоровался, но не попрощался"
250,3,1,manager,Алло дмитрий добрый день,True,"Менеджер поздоровался, но не попрощался"
251,3,2,manager,Добрый меня максим зовут компания китобизнес у...,True,ОК
300,3,51,manager,Угу все хорошо да понедельника тогда всего доб...,False,ОК
335,4,33,manager,Во вторник все ну с вами да тогда до вторника ...,False,"Менеджер попрощался, но не поздороваться"
479,5,142,manager,Ну до свидания хорошего вечера,False,"Менеджер попрощался, но не поздороваться"
